In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
from fuzzywuzzy import fuzz

In [2]:
gfd1 = pd.read_csv(r"...\GFD-1\LGC.txt", sep="\t", encoding='cp1252', dtype=str)
gfd1.head()

,LGCCode,LGCName
0,057002,Bond AUD
1,057003,Bond CAD
2,057004,Bond CAD Short Term
3,057005,Bond CHF
4,057006,Bond CHF Short Term


In [3]:
cfg = pd.read_csv(r"...\CFG\LGC.txt", sep="\t", dtype=str)
cfg_cols = cfg.columns
cfg.head()

,LGCCode,LGCName
0,058100,Equity Sweden Sm&Mid Cap
1,058097,Equity Argentina
2,058101,Equity Vietnam
3,058115,Equity Australia
4,059011,Hedge/Managed Futures/CTAs


In [4]:
cfg.equals(gfd1)

False

In [5]:
on=["LGCCode"]
df = cfg.merge(gfd1, how='outer', on=on, suffixes=("_CFG", "_GFD-1"))
df = df.fillna("-")
del cfg
del gfd1
display(df.sample(n=5))
text_cols = []

,LGCCode,LGCName_CFG,LGCName_GFD-1
394,057006,Bond CHF Short Term,Bond CHF Short Term
151,061012,Money Market JPY,Money Market JPY
47,057046,Bond JPY,Bond JPY
88,061026,Money Market NZD,Money Market NZD
93,063017,Absolute Return GBP Low,Absolute Return GBP Low


In [6]:
writer = pd.ExcelWriter("LGC Checks.xlsx", engine='xlsxwriter')
df_dict = {}
for col in cfg_cols:
    if col in on: continue
    print(col+"\n")
    check=[]
    ratio = []
    columns = []
    columns = on + [col+"_CFG", col+"_GFD-1"]
    #print(columns)
    check_df = df.loc[:, columns]
    for n in range(df.shape[0]):
        if col in text_cols:
            ratio.append(fuzz.ratio(str(check_df.iloc[n, -2]),str(check_df.iloc[n, -1])))
        if str(check_df.iloc[n, -2]) == str(check_df.iloc[n, -1]):
            check.append("Match")
        elif str(check_df.iloc[n, -2]) == "-":
            check.append("Data not in CFG")
        elif str(check_df.iloc[n, -1]) == "-":
            check.append("Data not in GFD-1")
        else:
            check.append("Difference")
    check_df["Check"] = np.array(check)
    if col in text_cols:
        check_df["Ratio"] = np.array(ratio)
    check_df.drop_duplicates(inplace=True)
    check_df.to_excel(writer,'%s' % col, index=False)
    check_df[check_df.Check != "Match"].to_excel(writer,'%s Diff' % col, index=False)
    display(check_df.sample(n=5))
writer.save()

LGCName



,LGCCode,LGCName_CFG,LGCName_GFD-1,Check
444,063035,Target Maturity MA USD 2050+,Target Maturity MA USD 2050+,Match
345,058011,Equity China,Equity China,Match
258,060104,Mixed Asset RUB Flexible,Mixed Asset RUB Flexible,Match
14,060038,Mixed Asset IDR Conservative,Mixed Asset IDR Conservative,Match
126,058089,Equity Israel,Equity Israel,Match
